The model tuning part is not focused here, I went through the tensorflow documentation to try to understand what batching, prefetching and other terms in the last cell of notebook means, and how to implement them.



In [2]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
import time
from tensorflow import keras

In [4]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


note: we are using pandas for reading and preprocessing

In [6]:
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [7]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes # categorical to int
df.thal.dtype

dtype('int8')

In [9]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,2,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,4,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [17]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [18]:
test_dataset.shape

(61, 14)

In [19]:
train_labels = train_dataset.pop('target')
test_labels = test_dataset.pop('target')

In [23]:
train_labels.shape

(242,)

In [24]:
def norm(x, train_stats):
  return (x- train_stats['mean']) / train_stats['std'] #normalize the data -- do this always (put 0 to 1)
# to ensure there is no bigger values, which cause problems in NN

train_stats = train_dataset.describe() # summary statistics
train_stats = train_stats.transpose()
train_stats # 

,count,mean,std,min,25%,50%,75%,max
age,242.0,54.797521,9.123650,29.0,48.00,56.0,62.00,77.0
sex,242.0,0.669421,0.471396,0.0,0.00,1.0,1.00,1.0
cp,242.0,3.070248,1.038283,0.0,2.00,3.0,4.00,4.0
trestbps,242.0,132.086777,18.128538,94.0,120.00,130.0,140.00,200.0
chol,242.0,245.727273,52.152505,126.0,211.00,240.0,273.75,564.0
fbs,242.0,0.132231,0.339444,0.0,0.00,0.0,0.00,1.0
restecg,242.0,1.024793,0.989260,0.0,0.00,1.5,2.00,2.0
thalach,242.0,149.016529,23.135353,88.0,132.25,152.0,165.00,202.0
exang,242.0,0.309917,0.463418,0.0,0.00,0.0,1.00,1.0
oldpeak,242.0,1.045041,1.135944,0.0,0.00,0.8,1.80,6.2


In [25]:
normed_train_data = norm(train_dataset, train_stats)# call the normalize fucn above
normed_train_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
225,0.351009,-1.420082,0.895470,-0.115110,-0.934323,-0.389553,-1.035919,-0.778745,-0.668765,-0.391781,0.687374,-0.744473,-0.444130
152,-1.073860,0.701275,0.895470,0.546830,1.213225,-0.389553,0.985794,-0.087162,1.489116,-0.919976,0.687374,2.434865,1.113543
228,0.022193,-1.420082,0.895470,-0.225433,-0.780926,-0.389553,-0.025063,-0.821968,1.489116,0.840674,0.687374,0.315306,1.113543
201,-1.293070,0.701275,0.895470,-0.942535,1.098178,-0.389553,-1.035919,1.382450,-0.668765,0.136414,0.687374,-0.744473,-0.444130
52,-0.197018,-1.420082,0.895470,-0.115110,0.350371,-0.389553,0.985794,-0.260058,-0.668765,-0.567846,0.687374,-0.744473,-0.444130
245,0.241403,0.701275,0.895470,-0.115110,-2.199842,-0.389553,-1.035919,-1.470327,1.489116,0.136414,0.687374,0.315306,1.113543
175,-0.087412,0.701275,0.895470,-1.218343,-0.761752,-0.389553,0.985794,-1.772894,1.489116,-0.919976,0.687374,0.315306,-0.444130
168,-0.087412,-1.420082,-0.067658,-1.328666,0.407895,-0.389553,0.985794,0.777316,-0.668765,-0.919976,-0.959601,-0.744473,-0.444130
223,-1.731491,-1.420082,-0.067658,0.326183,-0.493308,-0.389553,-1.035919,0.128957,-0.668765,-0.919976,0.687374,-0.744473,-0.444130
217,1.227851,0.701275,0.895470,1.539739,-0.339912,-0.389553,0.985794,-0.476177,-0.668765,1.104771,-0.959601,-0.744473,-2.001802


In [26]:
# normalize the test set too!
normed_test_data = norm(test_dataset, train_stats)# note the test_dataset and same train_stats
normed_test_data.head(10)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
1,1.337456,0.701275,0.895470,1.539739,0.772211,-0.389553,0.985794,-1.772894,1.489116,0.400511,0.687374,2.434865,-0.444130
9,-0.197018,0.701275,0.895470,0.436506,-0.819276,2.556439,0.985794,0.258629,1.489116,1.809031,2.334348,-0.744473,1.113543
17,-0.087412,0.701275,0.895470,0.436506,-0.128992,-0.389553,-1.035919,0.474748,-0.668765,0.136414,-0.959601,-0.744473,-0.444130
25,-0.525833,-1.420082,-0.067658,-0.666727,-0.512483,-0.389553,-1.035919,0.388301,-0.668765,0.488544,0.687374,-0.744473,-0.444130
28,-1.293070,0.701275,0.895470,0.988123,0.024404,-0.389553,-1.035919,0.950211,-0.668765,0.400511,-0.959601,-0.744473,-0.444130
31,0.570219,0.701275,0.895470,-0.832211,-0.301563,2.556439,-1.035919,0.474748,1.489116,0.312479,-0.959601,1.375085,1.113543
32,1.008640,0.701275,-0.067658,0.436506,1.711763,-0.389553,-1.035919,0.388301,-0.668765,-0.919976,-0.959601,-0.744473,-0.444130
35,-1.402676,0.701275,0.895470,0.436506,-0.378261,-0.389553,-1.035919,1.252778,-0.668765,-0.919976,-0.959601,-0.744473,-0.444130
38,0.022193,0.701275,0.895470,-0.004787,2.056905,-0.389553,-1.035919,-0.735521,1.489116,0.136414,0.687374,0.315306,1.113543
39,0.679824,0.701275,-0.067658,0.988123,-0.052294,2.556439,-1.035919,-0.519401,1.489116,-0.039651,0.687374,-0.744473,-0.444130


In [ ]:
# keras sequential API

In [27]:
model = keras.Sequential([
                          keras.layers.Dense(64, activation='relu', input_shape = [len(normed_train_data.keys())]),
                          keras.layers.Dense(64, activation='relu'),
                          keras.layers.Dense(1) # 1 because we have 1 output
])

model

In [28]:
optimizer = keras.optimizers.RMSprop(0.001)
model.compile(loss='mse',
              optimizer=optimizer,
              metrics= ['mse', 'mae'])
              

In [29]:
model.fit(normed_train_data, train_labels, epochs=10) # train_labels is y_train

Epoch 1/10
8/8 [==============================] - 0s 2ms/step - loss: 0.2200 - mse: 0.2200 - mae: 0.3366
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1292 - mse: 0.1292 - mae: 0.2765
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1129 - mse: 0.1129 - mae: 0.2531
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 0.1058 - mse: 0.1058 - mae: 0.2437
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0937 - mse: 0.0937 - mae: 0.2260
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0872 - mse: 0.0872 - mae: 0.2163
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0811 - mse: 0.0811 - mae: 0.2101
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0791 - mse: 0.0791 - mae: 0.2055
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 0.0760 - mse: 0.0760 - mae: 0.1996
Epoch 10/10
8/8 [==============================] - 0s 2

In [30]:
y_hats = model.predict(normed_test_data)

# y_hats # array of araays --> convert into singular numpy array
y_hats = [x[0] for x in y_hats]
keras.losses.MAE(test_labels, y_hats)

<tf.Tensor: shape=(), dtype=float32, numpy=0.31520057>

A more efficient way is to just use model.evaluate

In [31]:
model.evaluate(normed_test_data, test_labels) # returns a list

2/2 [==============================] - 0s 3ms/step - loss: 0.1769 - mse: 0.1769 - mae: 0.3152


[0.1768677830696106, 0.1768677830696106, 0.315200537443161]

# USING BATCHING AND PREFITTING
split data into multiple batch

eliminate bottlneck on model training by pre fetching data to save load, time

In [32]:
dataset = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values)) # input as numpy --> op as tf.data

In [33]:
dataset = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values)).batch(100) # splitts into 100 batches
# on the epoch it will loadup each individual batch sepreately on training.

In [34]:
model.fit(dataset, epochs=100) # fit the tensor

Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0607 - mse: 0.0607 - mae: 0.1803
Epoch 2/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0591 - mse: 0.0591 - mae: 0.1746
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0580 - mse: 0.0580 - mae: 0.1717
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0566 - mse: 0.0566 - mae: 0.1688
Epoch 5/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0551 - mse: 0.0551 - mae: 0.1655
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0535 - mse: 0.0535 - mae: 0.1622
Epoch 7/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0520 - mse: 0.0520 - mae: 0.1593
Epoch 8/100
3/3 [==============================] - 0s 3ms/step - loss: 0.0502 - mse: 0.0502 - mae: 0.1556
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.0486 - mse: 0.0486 - mae: 0.1520
Epoch 10/100
3/3 [============================

#  3/3 shows the number of batches loaded up
# 240 observations in train --> of 100 batch makes it 3 batches

# PREFETCH

In [35]:
dataset = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values)).batch(10).prefetch(2) # splitts into 100 batches
# on the epoch it will loadup each individual batch sepreately on training.

# it will prefetch 

In [36]:
model.fit(dataset, epochs=100)

Epoch 1/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0125 - mse: 0.0125 - mae: 0.0771
Epoch 2/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - mae: 0.0663
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0564
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - mae: 0.0612
Epoch 5/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0078 - mse: 0.0078 - mae: 0.0556
Epoch 6/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0069 - mse: 0.0069 - mae: 0.0511
Epoch 7/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0586
Epoch 8/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082 - mae: 0.0630
Epoch 9/100
25/25 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0536
Epoch 10/100
25/25 [========

In [ ]:
# The prev method of batch size had a little latency
# when it starts training it will go grab the next batch and load it up --> have data in memory ready to go
# prefetch(2) means 2 batches at a time
# have data ready to go
# stored in disk

# parellizing data extraction

In [ ]:
# data is stored in remote location eg S3, other location
# it will bottleneck the process
# pipeline will slow down --> thus use parallel processing

# time-to-first-byte ------> compute is far away then 
# takes maginitude of loading data longer 

# Read throughput ----> remote storage can have a lot of bandwidth, but for reading sequentially we are not using a lot of the bandwidth
# thus if we parallelize it then we can use all the available bandwith

# Interleave ---> tf transformation used to parellize the data loading step , can specify number of datasets to overlap and level of parellesim(how many threads)

# use tf.data.Dataset.interleave(dataset)
# default is 2 datasets, thus use "num_parellel_calls" run in parallel
# tf.data.experimental.autotune --> say how much paralellism you want to use when using runtime

# autotune -->look at threads, data available and load up as fast as possible
